In [ ]:
pip install pandas openpyxl

In [ ]:
from tkinter import filedialog
from tkinter import *
root = Tk()
root.withdraw()

import pandas as pd


In [ ]:
file_path_readers = filedialog.askopenfilename(title="Select file listing the readers allocations)")
print(file_path_readers)

In [ ]:

# Read the 'Students' sheet from the 'data' file into a pandas DataFrame
sheet_name = 'Final Allocations'
try:
    data_projects = pd.read_excel(file_path_readers, sheet_name=sheet_name)
    print(f"{sheet_name} data loaded successfully!")
except Exception as e:
    print(f"An error occurred while loading {sheet_name} data: {e}")
    

In [ ]:
import os
import openpyxl

#=== CREATE SPREADSHEET FOR STUDENTS AND SUPERVISORS ============
#----------------------------------------------------------------
# Create new DataFrame to save preferences as table

# Define the column names
results_columns = ['Staff Name', 'Count 1st Reader', 'Count 2nd Reader', 'Count Total']

print(results_columns)

# Create an empty DataFrame with specified columns
results_df = pd.DataFrame(columns=results_columns)


#----------------------------------------------------------------
# Add data to results DataFrame


stats = {}


for index, row in data_projects.iterrows():

    supervisor = row['Supervisor']
    reader = row['2nd Reader']
    
    if supervisor in stats:
        new_counts = [stats[supervisor][0]+1, stats[supervisor][1], stats[supervisor][2]+1]
        stats[supervisor] = new_counts 
    else:
        new_counts = [1,0,1]
        stats[supervisor] = new_counts 

    if reader in stats:
        new_counts = [stats[reader][0], stats[reader][1]+1, stats[reader][2]+1]
        stats[reader] = new_counts 
    else:
        new_counts = [0,1,1]
        stats[reader] = new_counts 

print(stats)

for name, stat in stats.items():

    result = pd.DataFrame({'Staff Name' : [name],
                           'Count 1st Reader' : [stat[0]],
                           'Count 2nd Reader' : [stat[1]], 
                           'Count Total' : [stat[2]]})
                
    results_df = pd.concat([results_df, result], ignore_index=True)


if 'NONE' in results_df.columns:
    results_df.drop(columns=['NONE'], inplace=True)

results_df = results_df.sort_values(by='Staff Name')

print(results_df)


In [ ]:
from openpyxl.worksheet.table import Table, TableStyleInfo

#Set path for the results XLSX file:
file_name_readers_noextension = os.path.splitext(os.path.basename(file_path_readers))[0]
file_name_results = file_name_readers_noextension + '_Statistics.xlsx'
print(file_name_results)

folder_results = os.path.dirname(os.path.abspath(file_path_readers))
file_path_results = os.path.join(folder_results,file_name_results)
print (file_path_results)

wb = openpyxl.Workbook()
wb.save(file_path_results)

# Use ExcelWriter to write to the specified filename
with pd.ExcelWriter(file_path_results, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    results_df.to_excel(writer, sheet_name='Final Allocations', index=False)

    # Load workbook and worksheet
    wb = writer.book
    ws = wb["Final Allocations"]

    # Define Table Range
    table = Table(displayName="Table1", ref=ws.dimensions)

    # Apply Table Style
    style = TableStyleInfo(name="TableStyleMedium9", showFirstColumn=False,
                           showLastColumn=False, showRowStripes=True, showColumnStripes=False)
    table.tableStyleInfo = style

    # Add table to worksheet
    ws.add_table(table)

    #wb.save(file_path_results)


print(f"Sheet 'Final Allocations' has been successfully written to {file_path_results}")

wb = openpyxl.load_workbook(file_path_results)
print(wb.sheetnames)
defaultSheet = wb['Sheet']
wb.remove(defaultSheet)
wb.save(file_path_results)